In [3]:
from __future__ import division
import pandas as pd
import numpy as np
#import matplotlib as plt
import datetime as dt
import pytz
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from scipy.cluster.hierarchy import dendrogram, linkage
from sklearn.cluster import AgglomerativeClustering
from time import time
from mpl_toolkits import mplot3d
import seaborn as sns
import matplotlib.pyplot as plt

from google.cloud import bigquery
client = bigquery.Client()
%load_ext google.cloud.bigquery

/Users/gunnar.griese/Desktop/env/lib/python2.7/site-packages/google/auth/_default.py:66: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK. We recommend that most server applications use service accounts instead. If your application continues to use end user credentials from Cloud SDK, you might receive a "quota exceeded" or "API not enabled" error. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)
/Users/gunnar.griese/Desktop/env/lib/python2.7/site-packages/google/auth/_default.py:66: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK. We recommend that most server applications use service accounts instead. If your application continues to use end user credentials from Cloud SDK, you might receive a "quota exceeded" or "API not enabled" error. For more information about service accounts, see https://cl

The google.cloud.bigquery extension is already loaded. To reload it, use:
  %reload_ext google.cloud.bigquery


In [8]:
%%bigquery df
#standardSQL

WITH query_settings AS (
  SELECT
    '20170501' AS start_date,
    '20170801' AS end_date), 

session_level AS (
SELECT 
fullVisitorId, 
CONCAT(fullVisitorId, CAST(visitId AS STRING)) AS session_id,
date,
visitNumber,
visitStartTime,
device.deviceCategory AS device,
geoNetwork.country AS country,
socialEngagementType,
totals.hits AS hits,
totals.pageviews AS pageviews,
IFNULL(totals.timeOnSite, 0) AS time_on_site,
IFNULL(totals.bounces, 0) AS bounces,
IFNULL(totals.transactions, 0) AS transactions,
IFNULL(totals.totalTransactionRevenue, 0) AS revenue,
totals.newVisits AS new_visitor,
totals.sessionQualityDim AS session_quality,
channelgrouping,
trafficSource.medium AS medium,
trafficSource.source AS source
FROM `bigquery-public-data.google_analytics_sample.ga_sessions_*`, query_settings 
WHERE _TABLE_SUFFIX BETWEEN start_date AND end_date),

hit_level AS (
SELECT session_id_p, MAX(landing_page) AS landing_page, MAX(exit_page) AS exit_page
FROM(
SELECT
CONCAT(fullVisitorId, CAST(visitStartTime AS STRING)) AS session_id_p,
FIRST_VALUE(hits.page.pagePath)
    OVER (PARTITION BY CONCAT(fullVisitorId, CAST(visitStartTime AS STRING)) ORDER BY hits.time ASC
    ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING) AS landing_page,
LAST_VALUE(hits.page.pagePath)
    OVER (PARTITION BY CONCAT(fullVisitorId, CAST(visitStartTime AS STRING)) ORDER BY hits.time ASC
    ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING) AS exit_page
FROM `bigquery-public-data.google_analytics_sample.ga_sessions_*` AS GA, query_settings, UNNEST(GA.hits) AS hits 
WHERE _TABLE_SUFFIX BETWEEN start_date AND end_date)
GROUP BY 1)

SELECT * EXCEPT(session_id_p)
FROM session_level AS sl
LEFT JOIN hit_level AS hl
ON sl.session_id = hl.session_id_p

,fullVisitorId,session_id,date,visitNumber,visitStartTime,device,country,socialEngagementType,hits,pageviews,...,bounces,transactions,revenue,new_visitor,session_quality,channelgrouping,medium,source,landing_page,exit_page
0,1381334533431476543,13813345334314765431494259348,20170508,1,1494259348,desktop,Argentina,Not Socially Engaged,5,5.0,...,0,0,0,1.0,NaN,Direct,(none),(direct),/home,/home
1,7545515601366144949,75455156013661449491494237224,20170508,2,1494237224,desktop,Singapore,Not Socially Engaged,26,26.0,...,0,0,0,NaN,NaN,Organic Search,organic,google,/home,/home
2,8236078121650217749,82360781216502177491494282823,20170508,1,1494282823,desktop,United States,Not Socially Engaged,5,5.0,...,0,0,0,1.0,NaN,Organic Search,organic,google,/home,/google+redesign/apparel/womens/womens+t+shirts
3,8282186549958913877,82821865499589138771494270851,20170508,1,1494270851,desktop,United States,Not Socially Engaged,9,9.0,...,0,0,0,1.0,NaN,Organic Search,organic,google,/home,/signin.html
4,442246838875347559,4422468388753475591494257463,20170508,1,1494257463,desktop,Egypt,Not Socially Engaged,6,6.0,...,0,0,0,1.0,NaN,Organic Search,organic,google,/home,/google+redesign/office/notebooks++journals
5,5672736312596936372,56727363125969363721494294021,20170508,1,1494294021,desktop,Taiwan,Not Socially Engaged,6,6.0,...,0,0,0,1.0,NaN,Organic Search,organic,google,/home,/google+redesign/office/notebooks++journals
6,9932451826989524671,99324518269895246711494263269,20170508,1,1494263269,desktop,Poland,Not Socially Engaged,8,8.0,...,0,0,0,1.0,NaN,Organic Search,organic,google,/home,/google+redesign/accessories
7,7011646678792606961,70116466787926069611494232093,20170508,1,1494232093,desktop,Russia,Not Socially Engaged,9,8.0,...,0,0,0,1.0,NaN,Referral,referral,siliconvalley.about.com,/home,/google+redesign/bags
8,6842911315484234382,68429113154842343821494252035,20170508,1,1494252035,desktop,Italy,Not Socially Engaged,10,7.0,...,0,0,0,1.0,NaN,Organic Search,organic,google,/home,/storeitem.html
9,4714774962658021735,47147749626580217351494285876,20170508,1,1494285876,mobile,Mexico,Not Socially Engaged,10,9.0,...,0,0,0,1.0,NaN,Organic Search,organic,google,/home,/google+redesign/apparel/headgear


In [10]:
df.shape

(203317, 21)